In [1]:
# 필요한 패키지 import
#from tqdm import trange
from bs4 import BeautifulSoup
import requests
import pandas as pd

import re

In [4]:
book_df = pd.DataFrame(columns=['ID', 'Title', 'Rating', 'Author', 'Price', 'Pdate', 'Publisher', 'Btype', 'Salseindex'])
book_df


,ID,Title,Rating,Author,Price,Pdate,Publisher,Btype,Salseindex


In [8]:
# 가져올 책 범위,ID.. 값 정의
start_id = 97000000 # 125190808
end_id = 97003000


In [9]:
# ID를 증가시키며 책 정보 크롤링
# 여기서 range를 trange로 바꾸면 된다.
for book_id in range(start_id, end_id + 1):
    url = f'http://www.yes24.com/Product/Goods/{book_id}'
    try:
        r = requests.get (url)
        if r.status_code == 200:
            html=r.text
            soup=BeautifulSoup(html, 'lxml') #  BeautifulSoup 클래스의 인스턴스 생성

            # title = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2').get_text()
            try:
                title = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2').get_text()
            except AttributeError:
                title = None # 선택한 요소가 없어서 발생하는 AttributeError 처리

            try:
                rating = soup.select_one('#spanGdRating > a > em').get_text() # .get_text().strip()
            except AttributeError:
                rating = None

            try:
                author = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a').get_text()
            except AttributeError:
                author = None

            btype = 'new'
            try:
                # if soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > strong > em').get_text() == "중고도서":
                if "중고도서" in soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > strong > em').get_text().lower():
                    price = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(1) > table > tbody > tr.accentRow > td > span > em').get_text()  # 판매가격
                    btype = 'old'
                else:
                    price = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(3) > table > tbody > tr:nth-child(2) > td > span > em').get_text()  # 판매가격
            except AttributeError:
                price = None

            try:
                pdate = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date').get_text()   # 발행일
            except AttributeError:
                pdate = None

            try:
                publisher = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a').get_text()  # 출판사
            except AttributeError:
                publisher = None

            try:
                numbers = re.findall(r'\d+', soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_sellNum').get_text().replace(',', ''))
                number = int(numbers[0]) if numbers else None
                salseindex = number # soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_sellNum').get_text()  # 판매지수
            except AttributeError:
                salseindex = None



            # book_df = book_df.append({'ID':book_id, 'Title':title, 'Author':author, 'Rating':rating, 'Price':price, 'Pdate':pdate, 'Publisher':publisher, 'Btype':btype}, ignore_index=True)
            new_row = pd.DataFrame({'ID': [book_id], 'Title': [title], 'Author': [author], 'Rating': [rating], 'Price': [price], 'Pdate': [pdate], 'Publisher': [publisher], 'Btype': [btype], 'Salseindex': [salseindex]})
            book_df = pd.concat([book_df, new_row], ignore_index=True)

        else:
            print('에러발생1')
    except requests.RequestException as e:
        # 에러 발생시....
        print('에러발생2')
# book_df
book_df.to_csv('book.csv', encoding='utf-8')
